# Coursework2 - PDE4433 - Ashwin Murali Thanalapati / M01037932

In [2]:
%pip install scikit-learn
%pip install tensorflow
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install scipy

import os
import tensorflow as tf
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from PIL import Image


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgr

In [11]:

# Function to load images from a folder
def load_images_from_folder(folder):
    images = []
    for root, dirs, files in os.walk(folder):
        for filename in files:
            img_path = os.path.join(root, filename)
            if img_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                img = Image.open(img_path)
            if img is not None:
                images.append(np.array(img))
    return images

# Load images from the 'terrains' folder
folder_path = 'Terrains'
images = load_images_from_folder(folder_path)

# Convert list of images to numpy array for training
images_array = np.array(images)
print(f"Loaded {len(images)} images from the folder.")

UnboundLocalError: cannot access local variable 'img' where it is not associated with a value

In [13]:
def load_images_from_folder(folder):
    images = []
    for root, dirs, files in os.walk(folder):
        for filename in files:
            img_path = os.path.join(root, filename)
            if img_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                img = Image.open(img_path)
                images.append(np.array(img))
    return images

# Load images from the 'terrains' folder
folder_path = 'Terrains'
images = load_images_from_folder(folder_path)

# Convert list of images to numpy array for training
images_array = np.asarray(images, dtype="object")
print(f"Loaded {len(images)} images from the folder.")

Loaded 3196 images from the folder.
